# Crash Carts
## PH 211 Lab
### February 07, 2021
### Zachary Bochanski

This lab uses data to describe how a simulated mobile student moves down a hallway. The idea is to plan for, the moving objects involved, to pass within close proximiety (1m) to each other and not collide. The carts are inteded to move at different constant velocitys with an acceptable, and planned margin of error.

In [2]:
### Imports

Importing packages `numpy` and `matplotlib` for use within the lab.

import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import matplotlib.patches as patch

In [7]:
### Simulated data sets
To keep things simple, sticking with group 1.

- Slow data: `group1`
- Fast data: `group1`

Hallway information:
- Hallway1: `6m` from intersection 
- Hallway2: `9m` from intersection 

In [ ]:
# Slow data group 1
slow_data = [0.94, 1.02, 0.92, 0.41, 1.23, 0.29] 

# Fast data group 1
fast_data = [2.49, 2.73, 1.96, 2.37, 1.52, 1.85]